# Identity model

In [ ]:
import graph_tool.all as gt
import numpy as np

gt.openmp_set_num_threads(1)

In [ ]:
def identity_model(group_size, l, b, alpha, z):
    '''
    Generates random identity networks as described by Watts, Dodds, & Newman 
    (https://doi.org/10.1126/science.1070120)
    
    group_size: int
        Size of groups at base level
    l: int
        Depth of hierarchy
    b: int
        Branching ratio
    alpha: double
        Assortativity parameter; when alpha >> 0, all inks will be as short as 
        possible; when alpha = -ln(b), the graph will be random
    z: double
        Average degree of final graph

    Outputs:

    group_membership: list
        Vector which node is in which group

    propensities: matrix
        Average number of edges between groups r and s
    '''
    propensities = np.zeros(shape=(b**(l-1), b**(l-1)))
    for i in range(l):
        for j in range(b**i):
            propensities[j*b**(l-(i+1)):(j+1)*b**(l-(i+1)), j*b**(l-(i+1)):(j+1)*b**(l-(i+1))] = (l-i)
    propensities = np.exp(-alpha*propensities)
    #propensities = 0.5/np.max(propensities)*propensities # 0.5 is the desired value in the diagonal
    group_membership = [item for row in [[group]*group_size for group in range(len(propensities))] for item in row]
    propensities = z*propensities/sum(sum(propensities/len(group_membership)))
    return group_membership, propensities

def detectable(group_membership, b):
    '''
    Returns the group membership list for the number of groups below the resolution limit
    '''
    n = len(group_membership)
    B = len(set(group_membership))
    B_max = len(group_membership)**0.5
    while B > B_max:
        B = B / b
    return [item for row in [[b]*int(n/B) for b in range(int(B))] for item in row]

In [ ]:
b = 2

group_membership, propensities = identity_model(
    group_size = 6, 
    l = 4, 
    b = b, 
    alpha = 2, 
    z = 4
)

print(f'Resolution limit: {int(len(group_membership)**0.5)} building blocks')
print('Using number of detectable building blocks\n')

print('Ground truth:')
g = gt.generate_sbm(group_membership, propensities)
detectable_group_membership_b = g.new_vp('int')
detectable_group_membership_b.a = detectable(group_membership, b)
pos = gt.sfdp_layout(g)
gt.graph_draw(g, pos=pos, vertex_fill_color=detectable_group_membership_b, output_size=(300, 300))

print('Inferred partition:')
state = gt.minimize_blockmodel_dl(g)
gt.graph_draw(g, pos=pos, vertex_fill_color=state.b, output_size=(300, 300))

print(f'Normalized mutual information: {round(gt.mutual_information(detectable(group_membership, b), state.b.a, norm=True), 2)}')
print(f'R: {round(gt.partition_overlap_center([detectable_group_membership_b.a, state.b.a], relabel_bs=False)[1], 2)}')